# Привет!

Перед началом прохождения убедитесь, что все настроено как надо. Следующая ячейка должна выполниться без ошибок:

После прохождения не забудьте **перезапустить ядро** (**restart the kernel**) (в панели меню выберите Kernel $\rightarrow$ Restart) и затем **запуск всех ячеек** (**run all cells**) (в панели меню выберите Cell $\rightarrow$ Run All).

Убедитесь, что заменили все такие места 
```python
# YOUR CODE HERE
raise NotImplementedError()
```
и
```
"YOUR ANSWER HERE"
```
на свои решения и ответы.

Если не установлены какие-то пакеты, то посмотрите нужные версии в ноутбуках по регрессии.

Для прорешивания используйте среды с поддержкой формата Jupyter: [Google Colab](https://colab.research.google.com/), [Jupyter Notebook (Web)](https://jupyter.org/), [PyCharm](https://www.jetbrains.com/ru-ru/pycharm/), [DataSpell](https://www.jetbrains.com/ru-ru/dataspell/) или [VSCode](https://code.visualstudio.com/). Работа с этим форматом в обычном текстовом редакторе может сыграть злую шутку.

Не забывайте о следующих особенностях:
- 😉 Помимо указанных тестов после задания существуют и скрытые тесты, поэтому внимательнее читайте задание
- 😎 Обязательно проверяйте выполнимость на тестах, для проверки перезапустите ядро и запустите все ячейки заново!
- 😂 После скачивания ноутбуков достаточно заполнить места для ответов и отправить ноутбук обратно (сам ноутбук, ни в архиве, ни ссылкой на GitHub ни как-то еще принимать не будем, загружаете сам файл)
- 😈 Не рекомендуем удалять исходные ячейки, так как из-за этого могут не начислиться баллы =(
  - 👼 Если таки такое случилось, скачайте заново шаблон и перенесите в него свои решения
- 😛 Свой код пишите только в ячейках для решений - любые дописывания в остальных ячейках (с текстом или тестами) могут не попасть в проверку (например, `import` вне ячейки для решения)


---

Это вторая контрольная работа с некоторыми вопросами по теме задачи регрессии в машинном обучении!

In [1]:
import numpy as np
print(f"numpy version: {np.__version__}")
import pandas as pd
print(f"pandas version: {pd.__version__}")
# Just to check that it exists
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")

from typing import Tuple, List, Dict

np.random.seed(123)

numpy version: 1.24.4
pandas version: 2.0.3
scikit-learn version: 1.3.2


### Задание 1

Напишите функцию для загрузки данных, которая возвращает tuple с DataFrame признаков и Series целевой переменной. Используйте [эту функцию](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html) в качестве основы для загрузки датасета диабетов.

> Это очень простая задача, но ее важно выполнить, так как она будет использоваться далее по тестам!

In [2]:
from sklearn.datasets import load_diabetes

def load_data() -> Tuple[pd.DataFrame, pd.Series]:
    loaded_data = load_diabetes()
    df = pd.DataFrame(loaded_data.data, columns=loaded_data.feature_names)
    df["target"] = loaded_data.target
    y = df.pop("target")

    return df, y

In [3]:
X, y = load_data()

np.testing.assert_array_equal(X.shape, [442, 10])
np.testing.assert_array_equal(y.shape, [442])

assert isinstance(X, pd.DataFrame)
assert isinstance(y, pd.Series)


### Задание 2

Напишите функцию поиска признаков в данных с корреляцией по Пирсону, превышающей порог по указанной переменной. Указанную переменную необходимо исключить.

Функция возвращает список имен переменных.

In [4]:
def get_high_correlation_features(df: pd.DataFrame, col_name: str, threshold: float) -> List[str]:
    popped = df.pop(col_name)
    corr_matrix = df.corrwith(popped)
    
    return corr_matrix[abs(corr_matrix) > threshold].index.tolist()

In [5]:
X, y = load_data()
df = X.copy()
df["target"] = y

_test_data = get_high_correlation_features(df, "target", threshold=0.27)
np.testing.assert_equal(
    _test_data, 
    np.array(['bmi', "bp", 's3', 's4', 's5', 's6'])
)


### Задание 3

Реализуйте класс для предобработки данных. Предобработка должна включать в себя:
- Определение 1го и 3го квартилей по указанному признаку и удаление данных за пределами квартилей по значениям признака. Удалить только данные строго больше 3го и строго меньше 1го квартилей.
- Масштабирование признаков к пределам [-1; 2]
- Опциональная чистка данных, подаваемых в transform

> Обратите внимание, на каких данных должен учиться scaler и в каком порядке должны идти операции в transform()

In [6]:
from sklearn.preprocessing import MinMaxScaler

class DataPreprocessing:
    def __init__(self, quartile_clean_col_name: str):
        self._col_name = quartile_clean_col_name
        self.q1 = None
        self.q3 = None
        self.scaler = MinMaxScaler((-1, 2))

    def fit(self, X: pd.DataFrame) -> None:
        self.q1 = X[self._col_name].quantile(0.25)
        self.q3 = X[self._col_name].quantile(0.75)
        X = X[
            (X[self._col_name] > self.q1) & 
            (X[self._col_name] < self.q3)
        ]
        self.scaler.fit_transform(X)
        

    def transform(self, X: pd.DataFrame, clean_data: bool = False) -> pd.DataFrame:
        if clean_data:
            X = X[
                (X[self._col_name] > self.q1) & 
                (X[self._col_name] < self.q3)
            ]
        X = self.scaler.transform(X)
        return pd.DataFrame(X, columns=self.scaler.feature_names_in_)

In [7]:
X, y = load_data()

_X_train_part = X.iloc[:350]
_X_test_part = X.iloc[350:]

preprocess = DataPreprocessing(quartile_clean_col_name='s2')
preprocess.fit(_X_train_part)
_X_test_prep = preprocess.transform(_X_test_part, clean_data=True)

assert isinstance(_X_test_prep, pd.DataFrame)
np.testing.assert_array_equal(_X_test_prep.shape, (44, 10))
np.testing.assert_array_equal(_X_test_prep.columns, X.columns)
assert 1.98 < _X_test_prep["s5"].max() < 2
assert _X_test_prep["s4"].max() > 1

np.testing.assert_almost_equal(_X_test_prep["bp"].mean(), 0.384, decimal=2)

# Check that index have to be saved from original data
assert 440 < _X_test_prep.index.max() <= 450
assert 350 <= _X_test_prep.index.min() < 360


AssertionError: 

### Задание 4

Реализуйте класс для обучения модели линейной регрессии с вычислением метрик. В метрики должны входить: MSE, RMSE, MAPE, $R^2$.

> Обратите внимание на возвращаемые типы данных: для `predict_evaluate` это пара значений, где второй аргумент - словарь с определёнными ключами.

> Класс должен содержать объект модели, методы которого вызываются в `fit()` и `predict_evaluate()`. 

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

class ModelTrainWithMetrics:
    def __init__(self):
        self.model = LinearRegression()

    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        self.model.fit(X, y)

    def predict_evaluate(self, X: pd.DataFrame, y: pd.Series) -> Tuple[np.ndarray, Dict[str, float]]:
        y_pred = self.model.predict(X)
        mse = mean_squared_error(y, y_pred)
        metrics = {
            'mse': mse,
            'rmse': np.sqrt(mse),
            'mape': np.mean(np.abs((y - y_pred) / y)) * 100,
            'r2': r2_score(y, y_pred),
        }
        return y_pred, metrics

In [9]:
from sklearn.model_selection import train_test_split

X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.3)

model = ModelTrainWithMetrics()
model.fit(X_train, y_train)
y_test_pred, metrics = model.predict_evaluate(X_test, y_test)

assert isinstance(metrics, dict)
assert isinstance(y_test_pred, np.ndarray)
np.testing.assert_equal(set(metrics.keys()), set(['mse', 'mape', 'rmse', 'r2']))

assert metrics['r2'] > 0.45
assert metrics['rmse'] < 55
print(f"MAPE = {metrics['mape']}")
assert metrics['mape'] < 0.37


MAPE = 36.67196318312674


AssertionError: 

### Задание 5

Реализуйте функцию сбора статистики данных датасета по каждой числовой колонке.

Формат выхода: колонки - показатели, строки - названия исходных колонок.

Требуемые показатели:
- Минимальное значение колонки
- Максимальное значение колонки
- Медианное значение колонки
- Среднее значение колонки / Mean value
- Стандартное (Среднеквадратическое) отклонение / Standart deviation
- Первый квартиль
- Третий квартиль
- Интерквартильный размах (IQR - Interquartile range)

In [ ]:
def collect_statistics(df: pd.DataFrame) -> pd.DataFrame:
    stat = df.describe()
    stat = stat.T
    stat["IQR"] = stat["75%"] - stat["25%"]
    stat = stat[["min", "25%", "50%", "mean", "std", "75%", "max", "IQR"]]
    stat.columns = ["min", "Q1", "median", "mean", "std", "Q3", "max", "IQR"]
    return stat

In [ ]:
X, _ = load_data()

stats = collect_statistics(X)

np.testing.assert_equal(set(stats.index), set(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']))
np.testing.assert_equal(set(stats.columns), set(['min', 'max', 'median', 'mean', 'std', 'Q1', 'Q3', 'IQR']))

np.testing.assert_almost_equal(stats.loc[['age', 'bmi'], 'mean'], [0, 0], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s3', 'bp'], 'std'], [0.048, 0.048], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s6', 's5'], 'min'], [-0.138, -0.126], decimal=2)
np.testing.assert_almost_equal(stats.loc[['s1', 's2'], 'max'], [0.154, 0.199], decimal=2)
